In [1]:
import ipaddress

In [6]:
ipadd = "141.4.60.63"
ip = ipadd.split(".")
print ('{0:08b}.{1:08b}.{2:08b}.{3:08b}'.format(int(ip[0]),int(ip[1]),int(ip[2]),int(ip[3])))

10001101.00000100.00111100.00111111


## CRC

In [3]:
def xor(a, b):
 
    # initialize result
    result = []
 
    # Traverse all bits, if bits are
    # same, then XOR is 0, else 1
    for i in range(1, len(b)):
        if a[i] == b[i]:
            result.append('0')
        else:
            result.append('1')
 
    return ''.join(result)
 
 
# Performs Modulo-2 division
def mod2div(divident, divisor):
 
    # Number of bits to be XORed at a time.
    pick = len(divisor)
 
    # Slicing the divident to appropriate
    # length for particular step
    tmp = divident[0 : pick]
 
    while pick < len(divident):
 
        if tmp[0] == '1':
 
            # replace the divident by the result
            # of XOR and pull 1 bit down
            tmp = xor(divisor, tmp) + divident[pick]
 
        else: # If leftmost bit is '0'
 
            # If the leftmost bit of the dividend (or the
            # part used in each step) is 0, the step cannot
            # use the regular divisor; we need to use an
            # all-0s divisor.
            tmp = xor('0'*pick, tmp) + divident[pick]
 
        # increment pick to move further
        pick += 1
 
    # For the last n bits, we have to carry it out
    # normally as increased value of pick will cause
    # Index Out of Bounds.
    if tmp[0] == '1':
        tmp = xor(divisor, tmp)
    else:
        tmp = xor('0'*pick, tmp)
 
    checkword = tmp
    return checkword
 
# Function used at the sender side to encode
# data by appending remainder of modular division
# at the end of data.
def encodeData(data, key):
 
    l_key = len(key)
 
    # Appends n-1 zeroes at end of data
    appended_data = data + '0'*(l_key-1)
    remainder = mod2div(appended_data, key)
 
    # Append remainder in the original data
    codeword = data + remainder
    return codeword


input_string = "10110111010101"
#s.sendall(input_string)
#data =(''.join(format(ord(x), 'b') for x in input_string))
data = input_string
print("Entered data in binary format :",data)
key = "11001"
 
ans = encodeData(data,key)
print("Encoded data to be sent to server in binary format :",ans)

Entered data in binary format : 10110111010101
Encoded data to be sent to server in binary format : 101101110101010101


## Hamming

In [7]:
# Python program to demonstrate
# hamming code


def calcRedundantBits(m):

	# Use the formula 2 ^ r >= m + r + 1
	# to calculate the no of redundant bits.
	# Iterate over 0 .. m and return the value
	# that satisfies the equation

	for i in range(m):
		if(2**i >= m + i + 1):
			return i

def posRedundantBits(data, r):

	# Redundancy bits are placed at the positions
	# which correspond to the power of 2.
	j = 0
	k = 1
	m = len(data)
	res = ''

	# If position is power of 2 then insert '0'
	# Else append the data
	for i in range(1, m + r+1):
		if(i == 2**j):
			res = res + '0'
			j += 1
		else:
			res = res + data[-1 * k]
			k += 1

	# The result is reversed since positions are
	# counted backwards. (m + r+1 ... 1)
	return res[::-1]


def calcParityBits(arr, r):
	n = len(arr)

	# For finding rth parity bit, iterate over
	# 0 to r - 1
	for i in range(r):
		val = 0
		for j in range(1, n + 1):

			# If position has 1 in ith significant
			# position then Bitwise OR the array value
			# to find parity bit value.
			if(j & (2**i) == (2**i)):
				val = val ^ int(arr[-1 * j])
				# -1 * j is given since array is reversed

		# String Concatenation
		# (0 to n - 2^r) + parity bit + (n - 2^r + 1 to n)
		arr = arr[:n-(2**i)] + str(val) + arr[n-(2**i)+1:]
	return arr


def detectError(arr, nr):
	n = len(arr)
	res = 0

	# Calculate parity bits again
	for i in range(nr):
		val = 0
		for j in range(1, n + 1):
			if(j & (2**i) == (2**i)):
				val = val ^ int(arr[-1 * j])

		# Create a binary no by appending
		# parity bits together.

		res = res + val*(10**i)

	# Convert binary to decimal
	return int(str(res), 2)


# Enter the data to be transmitted
data = '1110010'

# Calculate the no of Redundant Bits Required
m = len(data)
r = calcRedundantBits(m)

# Determine the positions of Redundant Bits
arr = posRedundantBits(data, r)

# Determine the parity bits
arr = calcParityBits(arr, r)

# Data to be transferred
print("Data transferred is " + arr)

# Stimulate error in transmission by changing
# a bit value.
# 10101001110 -> 11101001110, error in 10th position.

arr = '11001110000'
print("Error Data is " + arr)
correction = detectError(arr, r)
if(correction==0):
	print("There is no error in the received message.")
else:
	print("The position of error is ",len(arr)-correction+1,"from the left")


Data transferred is 11110011001
Error Data is 11001110000
The position of error is  7 from the left
